# Task 2: ResNet

In the following ResNet models for ECG heartbeat classification are trained. We trained different models from data from the [MIT-BIH Arrythmia Database](https://physionet.org/content/mitdb/1.0.0/) and [PTB Diagnostic ECG Database](https://physionet.org/physiobank/database/ptbdb/). 

First three different net-architectures are compared with a 5-fold cross validation in order to determine the best structure among these. ResNetSmall has 5 residual blocks and 16 filters in each layer, whereas ResNetStandard has 32 filters in each layer. ResNetDS is a ResNet model with downsampling. It consists of 7 residual blocks with an increasing number of filters.  

Second, the hyperparameters are tuned with a grid search. Due to computational reasons we did the grid search on the PTB data and then used the optimal parameters for the MIT-BIH data too. Then the final model is trained with all available training data and the classes for the test set are predicted. 

Further information can be found in the corresponding section of the report.

In [1]:
import numpy as np
import pandas as pd
import json
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from keras import losses, activations, models
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, average_precision_score
import sys
sys.path.append("../")
from models import *

# MITBIH Dataset

In [ ]:
# read data
df_train = pd.read_csv("../input/mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("../input/mitbih_test.csv", header=None)

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

In [ ]:
# determine best performing resnet architecture
accs = {
    "ResNet_small" : [],
    "ResNet_standard" : [],
    "ResNet_downsample" : []
}

for fold, (train, val) in enumerate(KFold(n_splits=5, shuffle=True,random_state = 123).split(X,Y)):
    file_path_small = f"Results/ResNet_small_MITBIH.h5"
    checkpoint_small = ModelCheckpoint(file_path_small, monitor='val_acc', verbose=1, save_best_only=True, mode='max')    
    early_small = EarlyStopping(monitor='val_acc', patience=7)
    callbacks_list_small = [checkpoint_small, early_small] 

    file_path_standard = f"Results/ResNet_standard_MITBIH.h5"
    checkpoint_standard = ModelCheckpoint(file_path_standard, monitor='val_acc', verbose=1, save_best_only=True, mode='max')    
    early_standard = EarlyStopping(monitor='val_acc', patience=7)
    callbacks_list_standard = [checkpoint_standard, early_standard] 

    file_path_ds = f"Results/ResNet_DS_MITBIH.h5"
    checkpoint_ds = ModelCheckpoint(file_path_ds, monitor='val_acc', verbose=1, save_best_only=True, mode='max')    
    early_ds = EarlyStopping(monitor='val_acc', patience=7)
    callbacks_list_ds = [checkpoint_ds, early_ds]

    resnet_small = ResNetSmall(5, optimizer=optimizers.Adam(0.001), callbacks=callbacks_list_small)
    resnet_standard = ResNetStandard(5,0.1,optimizer=optimizers.Adam(0.001), callbacks=callbacks_list_standard)
    resnet_ds = ResNetDS(5,optimizer=optimizers.Adam(0.001), callbacks=callbacks_list_ds)

    resnet_small.fit(X[train], Y[train], epochs=1, batch_size=128, verbose=1, validation_data = (X[val],Y[val]) )
    resnet_standard.fit(X[train], Y[train], epochs=1, batch_size=128, verbose=1, validation_data = (X[val],Y[val]) )
    resnet_ds.fit(X[train], Y[train], epochs=1, batch_size=128, verbose=1, validation_data = (X[val],Y[val]) )

    accs["ResNet_small"].append(resnet_small.score(X[val],Y[val]))
    accs["ResNet_standard"].append(resnet_standard.score(X[val],Y[val]))
    accs["ResNet_downsample"].append(resnet_ds.score(X[val],Y[val]))

    with open("Results/ResNet_CV_results_MITBIH.json", "w") as outfile:
        json.dump(accs, outfile)

In [ ]:
f = open('ResNet_CV_results_MITBIH.json')
results_mitbih = json.load(f)
for model in results_mitbih.keys():
print("Architecture", model)
mean_acc = sum(elt for elt in results_mitbih[model])/len(results_mitbih[model])
print(f"mean acc: {mean_acc}")

Best performing model: Architecture ResNet_Standard

mean loss: **0.05553194060921669** 	 mean acc: **0.9836557745933533**

## Train Final Model

In [ ]:
file_path_resnet = f"Results/ResNet_MITBIH.h5"
checkpoint_resnet = ModelCheckpoint(file_path_resnet, monitor='val_acc', verbose=1, save_best_only=True, mode='max')    
early_resnet = EarlyStopping(monitor='val_acc', patience=10)
callbacks_list_resnet = [checkpoint_resnet, early_resnet] 
resnet_model = ResNetStandard(5,0.1,optimizer=optimizers.RMSprop(0.001), callbacks=callbacks_list_resnet)
resnet_model.fit(X,Y, epochs=100, batch_size=128, verbose=2, validation_split=0.1 )

## Prediction

In [ ]:
resnet_model.load_weights("Results/ResNet_MITBIH.h5")
pred_test = resnet_model.predict(X_test)

f1 = f1_score(Y_test, pred_test,average="macro")
print(f"Test f1 score : {f1} ")
acc = accuracy_score(Y_test, pred_test)
print(f"Test accuracy : {acc} ")

# PTBDB data

In [ ]:
# read data
df_1 = pd.read_csv("../input/ptbdb_normal.csv", header=None)
df_2 = pd.read_csv("../input/ptbdb_abnormal.csv", header=None)
df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

In [ ]:
accs = {
    "init_small_ResNet" : [],
    "init_ResNet" : [],
    "downsample_ResNet" : []
}

for fold, (train, val) in enumerate(KFold(n_splits=5, shuffle=True,random_state = 123).split(X,Y)):
    
    file_path_small = f"Results/ResNet_small_PTBDB.h5"
    checkpoint_small = ModelCheckpoint(file_path_small, monitor='val_acc', verbose=0, save_best_only=True, mode='max')    
    early_small = EarlyStopping(monitor='val_acc', patience=7)
    callbacks_list_small = [checkpoint_small, early_small] 

    file_path_standard = f"Results/ResNet_standard_PTBDB.h5"
    checkpoint_standard = ModelCheckpoint(file_path_standard, monitor='val_acc', verbose=0, save_best_only=True, mode='max')    
    early_standard = EarlyStopping(monitor='val_acc', patience=7)
    callbacks_list_standard = [checkpoint_standard, early_standard] 

    file_path_ds = f"Results/ResNet_DS_PTBDB.h5"
    checkpoint_ds = ModelCheckpoint(file_path_ds, monitor='val_acc', verbose=0, save_best_only=True, mode='max')    
    early_ds = EarlyStopping(monitor='val_acc', patience=7)
    callbacks_list_ds = [checkpoint_ds, early_ds]

    resnet_small = ResNetSmall(1, optimizer=optimizers.Adam(0.001))
    resnet_standard = ResNetStandard(1,0.1,optimizer=optimizers.Adam(0.001))
    resnet_ds = ResNetDS(1,optimizer=optimizers.Adam(0.001))

    resnet_small.fit(X[train], Y[train], epochs=100, batch_size=128, verbose=1, callbacks=callbacks_list_small, validation_data = (X[val],Y[val]) )
    resnet_standard.fit(X[train], Y[train], epochs=100, batch_size=128, verbose=1, callbacks=callbacks_list_standard, validation_data = (X[val],Y[val]) )
    resnet_ds.fit(X[train], Y[train], epochs=100, batch_size=128, verbose=1, callbacks=callbacks_list_ds, validation_data = (X[val],Y[val]) )

    accs["ResNet_small"].append(resnet_small.score(X[val],Y[val]))
    accs["ResNet_standard"].append(resnet_standard.score(X[val],Y[val]))
    accs["ResNet_downsample"].append(resnet_ds.score(X[val],Y[val]))

    
    with open("Results/ResNet_CV_results_PTBDB.json", "w") as outfile:
        json.dump(accs, outfile)

In [ ]:
f = open('Results/ResNet_CV_results_PTBDB.json')
results_ptbdb = json.load(f)
means_ptbdb = []
for model in results_ptbdb.keys():
    print("Architecture", model)
    mean_acc = sum(elt for elt in results_ptbdb[model])/len(results_ptbdb[model])
    print(f"mean acc: {mean_acc}")

Best performing model: Architecture ResNet_Standard

mean loss: **0.042983403429389** 	 mean acc: **0.9849681377410888**

## Hyperparametertuning

In [ ]:
grid_search_results = {}

dropout_rate = [0.1,0.3]
opts = ["Adam", "SGD", "rmsprop"]
learning_rates = [("const", 0.0001), ("const", 0.001), ("exponential", 0.9), ("exponential", 0.1), ("poly", 1),("poly", 5)]

for dr in dropout_rate:
    for opt in opts: 
        for mode,factor in learning_rates:
            if mode == "const":
                lr = factor
            elif mode=="exponential": 
                lr = optimizers.schedules.ExponentialDecay(initial_learning_rate=0.005,decay_steps=1000,decay_rate=factor)
            else:
                lr = optimizers.schedules.PolynomialDecay(initial_learning_rate=0.01,decay_steps=1000,power=factor)
            if opt== "Adam":
                optimizer = optimizers.Adam(learning_rate = lr)
            elif opt == "SGD":
                optimizer = optimizers.SGD(learning_rate = lr)
            elif opt == "rmsprop":
                optimizer = optimizers.RMSprop(learning_rate = lr)
            
            grid_search_results[f"{dr}_{opt}_{mode}_{factor}"] = []
            
            for fold, (train, val) in enumerate(KFold(n_splits=3, shuffle=True,random_state = 123).split(X,Y)):
                print(f"FOLD: {fold} DR {dr} OPT {opt} MODDE {mode} FACTOR {factor}")
                file_path_standard = f"Results/ResNet_PTBDB_Hyperparam.h5"
                checkpoint_standard = ModelCheckpoint(file_path_standard, monitor='val_acc', verbose=0, save_best_only=True, mode='max')    
                early_standard = EarlyStopping(monitor='val_acc', patience=7)
                callbacks_list_standard = [checkpoint_standard, early_standard] 
                resnet_standard = ResNetStandard(1,dr,optimizer=optimizer, callbacks=callbacks_list_init)

                resnet_standard.fit(X[train], Y[train], epochs=1, batch_size=128, verbose=1, validation_data = (X[val],Y[val]))
    
                grid_search_results[f"{dr}_{opt}_{mode}_{factor}"].append(resnet_standard.score(X[val],Y[val]))

                with open("Results/ResNet_Hyperparam_CV_3_results_ptbdb.json", "w") as outfile:
                    json.dump(grid_search_results, outfile)

In [8]:
f = open('Results/ResNet_Hyperparam_CV_3_results_ptbdb.json')
grid_search_results = json.load(f)
means = []
for k in grid_search_results.keys():
    mean_acc = sum(elt for elt in grid_search_results[k])/len(grid_search_results[k])
    means.append((k, mean_acc))
grid_search_results
means.sort(reverse = True)
means[0]

('0.1_Adam_const_0.0001', 0.5779443159068189)

Based on the grid search the optimal parameters for the model are: 

dropout rate: **0.1** 

optimizer: **RMSProp**

learning rate: **constant of 0.001***

These parameters obtained performances of (average loss,  average accuracy): 

**(0.056621051083008446, 0.9811009367307028)**

## Train final model

In [ ]:
file_path_standard = f"Results/ResNet_PTBDB.h5"
checkpoint_standard = ModelCheckpoint(file_path_standard, monitor='val_acc', verbose=0, save_best_only=True, mode='max')    
early_standard = EarlyStopping(monitor='val_acc', patience=7)
callbacks_list_standard = [checkpoint_standard, early_standard] 
resnet_standard = ResNetStandard(1,dr,optimizer=optimizer, callbacks=callbacks_list_standard)
resnet_standard.fit(X, Y, epochs=100, batch_size=128, verbose=2, validation_split=0.1)

## Predict

In [ ]:
resnet_standard.load_weights("Results/ResNet_PTBDB.h5")
pred_test = resnet_standard.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy : %s "% acc)

auroc = roc_auc_score(Y_test, pred_test)

print("Test AUROC : %s "% auroc)

auprc = average_precision_score(Y_test, pred_test)

print("Test AUPRC : %s "% auprc)